In [ ]:
!pip install keybert

In [3]:
import numpy as np
import pandas as pd
import re
from keybert import KeyBERT

In [80]:
df = pd.read_csv("results_254_to_299.csv")

In [81]:
df.drop_duplicates(inplace=True)

In [82]:
df.head(10)

,HS_Code,Description
0,540792,WOVEN FABRIC DYED
1,540792,DACRON DYED WOVEN FABRIC
2,540792,WOVEN AND DYED FABRIC
3,540792,"EDIP PLAIN,,SILKEN, CELESTIAL-100 POLYESTER ED..."
10,540792,FURNISHING FABRICS COMMERCIAL INVOICE NO: EXGM...
12,540792,MACHINE PROCESSED PAPER WOVEN FABRIC DYED HTS.
15,540792,WOVEN FABRIC DYED 5407.61
16,540792,MINI MATT POLYESTER 100 HS CODE5407.92 .
17,540792,MINI MATT POLYESTER 100 HS CODE 5407.92 .
18,540792,WOVEN FABRICS HTS CODE: 5407.92


In [83]:
df['Description'] = df['Description'].apply(lambda x: x.split('HS CODE')[0])
df['Description'] = df['Description'].apply(lambda x: x.split('HTS CODE')[0])
df['Description'] = df['Description'].apply(lambda x: x.split('HSCODE')[0])
df['Description'] = df['Description'].apply(lambda x: x.split('H S CODE')[0])
df['Description'] = df['Description'].apply(lambda x: x.split('HS-CODE')[0])
df['Description'] = df['Description'].apply(lambda x: x.split('HS.CODE')[0])
df['Description'] = df['Description'].apply(lambda x: x.split('H.S.CODE')[0])
df['Description'] = df['Description'].apply(lambda x: x.split('HTSCODE')[0])
df['Description'] = df['Description'].apply(lambda x: x.split('HTS-CODE')[0])
df['Description'] = df['Description'].apply(lambda x: x.split('HTS.CODE')[0])
df['Description'] = df['Description'].apply(lambda x: re.sub(r'\d{4}\.\d{2}','', x))
df['Description'] = df['Description'].apply(lambda x: re.sub(r'\d{6}|\d{10}','', x))
df['Description'] = df['Description'].str.replace('\W', ' ')

In [84]:
df.head(10)

,HS_Code,Description
0,540792,WOVEN FABRIC DYED
1,540792,DACRON DYED WOVEN FABRIC
2,540792,WOVEN AND DYED FABRIC
3,540792,EDIP PLAIN SILKEN CELESTIAL 100 POLYESTER ED...
10,540792,FURNISHING FABRICS COMMERCIAL INVOICE NO EXGM...
12,540792,MACHINE PROCESSED PAPER WOVEN FABRIC DYED HTS
15,540792,WOVEN FABRIC DYED
16,540792,MINI MATT POLYESTER 100
17,540792,MINI MATT POLYESTER 100
18,540792,WOVEN FABRICS


In [85]:
kw_model = KeyBERT('distilbert-base-nli-mean-tokens')

In [86]:
df['Keyword_with_Scores'] = df['Description'].apply(lambda x: kw_model.extract_keywords(x,keyphrase_ngram_range=(1, 2), stop_words='english', use_maxsum=True, nr_candidates=20, top_n=5))

In [87]:
df.head(10)

,HS_Code,Description,Keyword_with_Scores
0,540792,WOVEN FABRIC DYED,"[(fabric, 0.818), (dyed, 0.8403), (woven, 0.85..."
1,540792,DACRON DYED WOVEN FABRIC,"[(dacron, 0.5351), (fabric, 0.7264), (woven fa..."
2,540792,WOVEN AND DYED FABRIC,"[(dyed, 0.8317), (fabric, 0.8389), (woven, 0.8..."
3,540792,EDIP PLAIN SILKEN CELESTIAL 100 POLYESTER ED...,"[(2021, 0.311), (cotton inv, 0.3298), (4195 st..."
10,540792,FURNISHING FABRICS COMMERCIAL INVOICE NO EXGM...,"[(exgms, 0.3268), (commercial invoice, 0.493),..."
12,540792,MACHINE PROCESSED PAPER WOVEN FABRIC DYED HTS,"[(hts, 0.3836), (paper, 0.5459), (machine proc..."
15,540792,WOVEN FABRIC DYED,"[(fabric, 0.818), (dyed, 0.8403), (woven, 0.85..."
16,540792,MINI MATT POLYESTER 100,"[(matt, 0.3134), (mini matt, 0.4233), (100, 0...."
17,540792,MINI MATT POLYESTER 100,"[(matt, 0.3134), (mini matt, 0.4233), (100, 0...."
18,540792,WOVEN FABRICS,[]


In [88]:
def extract_keywords(keyword_score_list):
    phrase_list = []
    if len(keyword_score_list) > 0:
        for keyword in keyword_score_list:
            if keyword[1] > 0.5:
                phrase_list.append(keyword[0])
        phrase_non_unique = " ".join(phrase_list)
        unique_words = set(phrase_non_unique.split(' '))
        phrase = " ".join(unique_words)

        return phrase
    else:
        return ""

In [89]:
df['Keywords'] = df['Keyword_with_Scores'].apply(extract_keywords)

In [90]:
df.head(10)

,HS_Code,Description,Keyword_with_Scores,Keywords
0,540792,WOVEN FABRIC DYED,"[(fabric, 0.818), (dyed, 0.8403), (woven, 0.85...",woven dyed fabric
1,540792,DACRON DYED WOVEN FABRIC,"[(dacron, 0.5351), (fabric, 0.7264), (woven fa...",woven dyed dacron fabric
2,540792,WOVEN AND DYED FABRIC,"[(dyed, 0.8317), (fabric, 0.8389), (woven, 0.8...",woven dyed fabric
3,540792,EDIP PLAIN SILKEN CELESTIAL 100 POLYESTER ED...,"[(2021, 0.311), (cotton inv, 0.3298), (4195 st...",100 polyester
10,540792,FURNISHING FABRICS COMMERCIAL INVOICE NO EXGM...,"[(exgms, 0.3268), (commercial invoice, 0.493),...",fabrics commercial furnishing 22
12,540792,MACHINE PROCESSED PAPER WOVEN FABRIC DYED HTS,"[(hts, 0.3836), (paper, 0.5459), (machine proc...",machine dyed hts processed fabric paper
15,540792,WOVEN FABRIC DYED,"[(fabric, 0.818), (dyed, 0.8403), (woven, 0.85...",woven dyed fabric
16,540792,MINI MATT POLYESTER 100,"[(matt, 0.3134), (mini matt, 0.4233), (100, 0....",100 polyester
17,540792,MINI MATT POLYESTER 100,"[(matt, 0.3134), (mini matt, 0.4233), (100, 0....",100 polyester
18,540792,WOVEN FABRICS,[],


In [91]:
df['Keywords'] = np.where(df['Keyword_with_Scores'].str.len() == 0, df['Description'].str.lower(), df['Keywords'])

In [92]:
df.head(10)

,HS_Code,Description,Keyword_with_Scores,Keywords
0,540792,WOVEN FABRIC DYED,"[(fabric, 0.818), (dyed, 0.8403), (woven, 0.85...",woven dyed fabric
1,540792,DACRON DYED WOVEN FABRIC,"[(dacron, 0.5351), (fabric, 0.7264), (woven fa...",woven dyed dacron fabric
2,540792,WOVEN AND DYED FABRIC,"[(dyed, 0.8317), (fabric, 0.8389), (woven, 0.8...",woven dyed fabric
3,540792,EDIP PLAIN SILKEN CELESTIAL 100 POLYESTER ED...,"[(2021, 0.311), (cotton inv, 0.3298), (4195 st...",100 polyester
10,540792,FURNISHING FABRICS COMMERCIAL INVOICE NO EXGM...,"[(exgms, 0.3268), (commercial invoice, 0.493),...",fabrics commercial furnishing 22
12,540792,MACHINE PROCESSED PAPER WOVEN FABRIC DYED HTS,"[(hts, 0.3836), (paper, 0.5459), (machine proc...",machine dyed hts processed fabric paper
15,540792,WOVEN FABRIC DYED,"[(fabric, 0.818), (dyed, 0.8403), (woven, 0.85...",woven dyed fabric
16,540792,MINI MATT POLYESTER 100,"[(matt, 0.3134), (mini matt, 0.4233), (100, 0....",100 polyester
17,540792,MINI MATT POLYESTER 100,"[(matt, 0.3134), (mini matt, 0.4233), (100, 0....",100 polyester
18,540792,WOVEN FABRICS,[],woven fabrics


In [93]:
hs_df = pd.read_csv("HSCodeData.csv", dtype={'id': object})

In [94]:
hs_df.head(10)

,Unnamed: 0,id,text,parent
0,0,010121,"Horses; live, pure",101
1,1,010129,"Horses; live, other than pure",101
2,2,010130,Asses; live,101
3,3,010190,Mules and hinnies; live,101
4,4,010221,"Cattle; live, pure",102
5,5,010229,"Cattle; live, other than pure",102
6,6,010231,"Buffalo; live, pure",102
7,7,010239,"Buffalo; live, other than pure",102
8,8,010290,"Bovine animals; live, other than cattle and b...",102
9,9,010310,"Swine; live, pure",103


In [95]:
hs_df['text'] = hs_df['text'].str.replace('\W', ' ')

In [96]:
hs_df.head(10)

,Unnamed: 0,id,text,parent
0,0,010121,Horses live pure,101
1,1,010129,Horses live other than pure,101
2,2,010130,Asses live,101
3,3,010190,Mules and hinnies live,101
4,4,010221,Cattle live pure,102
5,5,010229,Cattle live other than pure,102
6,6,010231,Buffalo live pure,102
7,7,010239,Buffalo live other than pure,102
8,8,010290,Bovine animals live other than cattle and b...,102
9,9,010310,Swine live pure,103


In [97]:
def jaccard_similarity(str1, str2):
    set1 = set(str1.split(' '))
    set2 = set(str2.split(' '))
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union)

In [98]:
best_matches = []
best_codes = []

for phrase in df['Keywords']:
    best_match = None
    best_similarity = 0
    best_code = None

    for sentence, hs_code in zip(hs_df['text'], hs_df['id']):
        similarity = jaccard_similarity(phrase, sentence.lower())
        if similarity > best_similarity:
            best_similarity = similarity
            best_match = sentence
            best_code = hs_code

    best_matches.append(best_match)
    best_codes.append(best_code)

# Add the best matching sentences to df_phrases
df['BestMatch'] = best_matches
df['BestCode'] = best_codes

In [99]:
df.head(10)

,HS_Code,Description,Keyword_with_Scores,Keywords,BestMatch,BestCode
0,540792,WOVEN FABRIC DYED,"[(fabric, 0.818), (dyed, 0.8403), (woven, 0.85...",woven dyed fabric,Fabrics woven 3,551312
1,540792,DACRON DYED WOVEN FABRIC,"[(dacron, 0.5351), (fabric, 0.7264), (woven fa...",woven dyed dacron fabric,Fabrics woven 3,551312
2,540792,WOVEN AND DYED FABRIC,"[(dyed, 0.8317), (fabric, 0.8389), (woven, 0.8...",woven dyed fabric,Fabrics woven 3,551312
3,540792,EDIP PLAIN SILKEN CELESTIAL 100 POLYESTER ED...,"[(2021, 0.311), (cotton inv, 0.3298), (4195 st...",100 polyester,Textile fabrics tyrecord of high tenacity ya...,590220
10,540792,FURNISHING FABRICS COMMERCIAL INVOICE NO EXGM...,"[(exgms, 0.3268), (commercial invoice, 0.493),...",fabrics commercial furnishing 22,Fabrics woven 3,551312
12,540792,MACHINE PROCESSED PAPER WOVEN FABRIC DYED HTS,"[(hts, 0.3836), (paper, 0.5459), (machine proc...",machine dyed hts processed fabric paper,Machine,845611
15,540792,WOVEN FABRIC DYED,"[(fabric, 0.818), (dyed, 0.8403), (woven, 0.85...",woven dyed fabric,Fabrics woven 3,551312
16,540792,MINI MATT POLYESTER 100,"[(matt, 0.3134), (mini matt, 0.4233), (100, 0....",100 polyester,Textile fabrics tyrecord of high tenacity ya...,590220
17,540792,MINI MATT POLYESTER 100,"[(matt, 0.3134), (mini matt, 0.4233), (100, 0....",100 polyester,Textile fabrics tyrecord of high tenacity ya...,590220
18,540792,WOVEN FABRICS,[],woven fabrics,Fabrics woven 3,551312


In [100]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [110]:
def accuracy(y_true, y_pred):

    correct_predictions = 0

    for yt, yp in zip(y_true, y_pred):

        if yt == yp:

            correct_predictions += 1

    #returns accuracy
    return correct_predictions / len(y_true)

In [111]:
accuracy(df['HS_Code'], df['BestCode'])

0.0493226814866273

In [112]:
df.loc[df['HS_Code'] == df['BestCode']]

,HS_Code,Description,Keyword_with_Scores,Keywords,BestMatch,BestCode
203,540792,WOVEN FABRICS OF SYNTHETIC FILAMNET YARNS DYED,"[(synthetic, 0.4913), (filamnet, 0.5334), (syn...",fabrics yarns synthetic dyed filamnet woven,Fabrics woven of synthetic filament yarn d...,540792
727,540810,100 VISCOSE WOVEN LADIES SKIRT,"[(viscose, 0.3939), (100, 0.5483), (woven ladi...",skirt 100 ladies viscose woven,Fabrics woven of viscose rayon high tenaci...,540810
728,540810,100 VISCOSE WOVEN JUNIORGIRLS,"[(woven, 0.2295), (viscose, 0.3703), (juniorgi...",100 juniorgirls viscose woven,Fabrics woven of viscose rayon high tenaci...,540810
729,540810,100 VISCOSE WOVEN GIRLS PULL ON SHORTS,"[(shorts, 0.3629), (100, 0.3979), (girls pull,...",100 pull viscose girls woven,Fabrics woven of viscose rayon high tenaci...,540810
730,540810,VISCOSE WOVEN TROUSERS,"[(woven, 0.6253), (viscose, 0.6547), (viscose ...",trousers viscose woven,Fabrics woven of viscose rayon high tenaci...,540810
...,...,...,...,...,...,...
5462,550410,VISCOSE STAPLE FIBER 3DX60MM,"[(staple, 0.4369), (viscose, 0.5795), (staple ...",fiber 3dx60mm staple viscose,Fibres artificial staple fibres of viscose ...,550410
5526,550410,VISCOSE STAPLE FIBER 3DX60MM FR PO,"[(fr po, 0.2555), (fr, 0.2778), (staple fiber,...",fiber 3dx60mm staple viscose,Fibres artificial staple fibres of viscose ...,550410
6019,550510,WASTE INCLUDING NOILS YARN WASTE AND GARNETT...,"[(including, 0.363), (pa66, 0.5137), (includin...",including garnetted pa66 noils waste yarn,Fibres waste including noils yarn waste an...,550510
6027,550510,WASTE INCLUDING NOILS YARN WASTE AND GARNETT...,"[(pressed, 0.2207), (pa66, 0.4378), (including...",including garnetted noils waste yarn,Fibres waste including noils yarn waste an...,550510
